In [1]:
def parse_input(file):
    with open(file) as file_in:
        numbers = file_in.read().splitlines()

    numbers = tuple(map(int, numbers))
    return numbers

In [2]:
def mix(value, secret):
    return value ^ secret

In [3]:
def prune(secret):
    return secret % 16777216

In [4]:
def get_next_secret(secret):
    secret = prune(mix(secret * 64, secret))
    secret = prune(mix(int(secret / 32), secret))
    secret = prune(mix(secret * 2048, secret))
    return secret

In [5]:
def get_nth_secret(secret, n):
    for __ in range(n):
        secret = get_next_secret(secret)
    return secret

In [13]:
def main1(file):
    buyers_secrets = parse_input(file)

    result = 0
    for secret in buyers_secrets:
        result += get_nth_secret(secret, n=2000)

    return result